# Module 7: Unit Test Concepts via GitHub Copilot (No-Code Mapping)

## Overview
Use **GitHub Copilot** conceptually to design manual test cases inspired by unit tests for `promotion_pricing_engine.py`.

### Objectives
- How Copilot generates unit tests (concept-level walkthrough).
- Convert unit test logic to manual test cases with prompt engineering.
- Design oracles and expected outcomes with LLM prompts.

**Hands-on Exercise:** Generate manual test cases modeled on Copilot-style unit test outputs for the pricing engine.

## Task 1 — Understand Copilot’s Unit Test Style
**Prompt:**  
"Explain how Copilot typically structures unit tests for a pricing function: setup, action, assertions, and edge-case parametrization."


**Answer**  

1. Setup (Arrange)
Prepare all necessary inputs and configurations:

Items list: Create representative Item objects (e.g., general items, food items, tax-exempt).
Quote object: Define region, membership level, and coupon codes.
Expected outputs: Precompute expected subtotal, discounts, tax, shipping, and total.

Example:

items = [Item(sku="A1", price=20.0, qty=2), Item(sku="B1X", price=10.0, qty=2)]
quote = Quote(region="CA", membership="GOLD", coupon_codes=["PCT10"])

2. Action (Act)
Call the function under test:

result = price_quote(items, quote)

3. Assertions
Verify correctness of:

Subtotal: Matches sum of item prices.
Discount logic: Membership + coupon applied correctly, respecting stacking and caps.
Shipping: Waived for GOLD or threshold.
Tax: Regional and category-specific rules.
Total: Includes rounding and min payable enforcement.
Reason flags: Contains expected flags like "membership_gold", "free_shipping", etc.

Example:

assert result.total == 1.0  # if min payable enforced
assert "discount_cap_applied" in result.reason_flags

4. Edge-Case Parametrization
Use pytest parametrize or similar to cover:

Regions: CA, NY, TX, INTL.
Membership tiers: None, SILVER, GOLD.
Coupons: PCT10, FLAT5, BOGO1, invalid codes.
Stacking limits: More than 2 discounts.
Boundary conditions:

Subtotal just below/above free shipping threshold.
Max discount cap (60%).
Tax-exempt and food items.
Total dropping below $1 (min payable rule).



Example:

@pytest.mark.parametrize("membership,subtotal,expected_discount", [
    ("GOLD", 100.0, 5.0),
    ("SILVER", 49.0, 0.0),
    ("SILVER", 50.0, 1.0),
])
def test_membership_discount(membership, subtotal, expected_discount):


**What happens:** You gain a pattern to mirror unit tests as manual cases.


## Task 2 — Convert a Copilot Test to Manual Case
**Prompt:**  
"Convert a GOLD + PCT10 scenario into a manual test with inputs and oracles."


**Answer**  

Test Case: GOLD Membership + PCT10 Coupon
Purpose
Verify that the pricing engine correctly applies:

Membership discount (GOLD = 5%)
Coupon discount (PCT10 = 10%)
Stacking rules (max 2 discounts)
Shipping waived for GOLD
Tax applied per region
Rounding and min payable guardrails


Inputs

Items:

SKU: A1, Price: $50.00, Qty: 1, Category: general
SKU: B1X, Price: $25.00, Qty: 1, Category: general


Quote:

Region: CA
Membership: GOLD
Coupon Codes: ["PCT10"]




Expected Oracles


Subtotal:
$50.00 + $25.00 = $75.00


Discounts:

Membership (GOLD): 5% of $75.00 = $3.75
Coupon (PCT10): 10% of $75.00 = $7.50
Total discount = $3.75 + $7.50 = $11.25
Check discount cap: 60% of $75.00 = $45.00 → Not exceeded



Subtotal after discounts:
$75.00 - $11.25 = $63.75


Shipping:
GOLD → $0.00 (free shipping)


Tax:
Region CA rate = 8.25%
Tax = $75.00 × 0.0825 = $6.19 (rounded)


Total:
$63.75 + $0.00 + $6.19 = $69.94


Reason Flags:
["membership_gold", "free_shipping"]



Pass Criteria

subtotal == 75.00
total == 69.94
Discounts list contains membership and coupon entries
Reason flags include "membership_gold" and "free_shipping"



**What happens:** You obtain a ready manual test mirroring unit assertions.


## Task 3 — Stacking Limit = 2
**Prompt:**  
"Create manual cases to verify only two discounts apply when more are supplied or when membership counts as one."


**Answer**  

Case 1: GOLD + PCT10 + FLAT5 (Three Discounts Supplied)

Inputs:

Items:

SKU: A1, Price: $50.00, Qty: 1
SKU: B1X, Price: $25.00, Qty: 1


Quote:

Region: CA
Membership: GOLD
Coupon Codes: ["PCT10", "FLAT5"]




Expected Oracles:

Subtotal = $75.00
Discounts applied:

Membership (GOLD): $3.75
Coupon (PCT10): $7.50
FLAT5 ignored (stacking limit reached)


Reason flags include: ["membership_gold", "stacking_limit_reached"]
Total discount = $11.25
Shipping waived (GOLD)
Tax = $6.19
Total = $69.94




Case 2: SILVER + PCT10 + FLAT5 (Membership Counts as One)

Inputs:

Items:

SKU: A1, Price: $60.00, Qty: 1
SKU: B1X, Price: $20.00, Qty: 1


Quote:

Region: NY
Membership: SILVER
Coupon Codes: ["PCT10", "FLAT5"]




Expected Oracles:

Subtotal = $80.00
Discounts applied:

Membership (SILVER): $1.60 (2% since subtotal ≥ $50)
Coupon (PCT10): $8.00
FLAT5 ignored


Reason flags include: ["membership_silver", "stacking_limit_reached"]
Shipping waived (subtotal ≥ $75)
Tax = $7.04
Total = $87.04 - $9.60 = $77.44




Case 3: No Membership + Three Coupons

Inputs:

Items:

SKU: A1, Price: $40.00, Qty: 1
SKU: B1X, Price: $20.00, Qty: 1


Quote:

Region: TX
Membership: None
Coupon Codes: ["PCT10", "FLAT5", "BOGO1"]




Expected Oracles:

Subtotal = $60.00
Discounts applied:

Coupon (PCT10): $6.00
Coupon (FLAT5): $5.00
BOGO1 ignored


Reason flags include: ["stacking_limit_reached"]
Shipping = $5.99 (subtotal < $75, no membership)
Tax = $3.75
Total = $60.00 - $11.00 + $5.99 + $3.75 = $58.74

**What happens:** You validate stacking enforcement with explicit oracles.


## Task 4 — Discount Cap 60%
**Prompt:**  
"Ensure discounts never exceed 60% of subtotal."


**Answer**  

Case 4: GOLD + PCT50 + FLAT20 (Cap Triggered)

Inputs:

Items:

SKU: A1, Price: $100.00, Qty: 1


Quote:

Region: CA
Membership: GOLD
Coupon Codes: ["PCT50", "FLAT20"]




Expected Oracles:

Subtotal = $100.00
Raw discounts:

Membership: $5.00
PCT50: $50.00
FLAT20 ignored (stacking limit reached)
Total raw discount = $55.00


Cap = 60% of $100.00 = $60.00 → Not exceeded yet
Total discount applied = $55.00
Shipping waived (GOLD)
Tax = $8.25
Total = $100.00 - $55.00 + $8.25 = $53.25
Reason flags: ["membership_gold", "free_shipping"]




Case 5: GOLD + PCT50 + PCT30 (Cap Enforced)

Inputs:

Items:

SKU: A1, Price: $200.00, Qty: 1


Quote:

Region: NY
Membership: GOLD
Coupon Codes: ["PCT50", "PCT30"]




Expected Oracles:

Subtotal = $200.00
Raw discounts:

Membership: $10.00
PCT50: $100.00
PCT30 ignored (stacking limit reached)
Total raw discount = $110.00


Cap = 60% of $200.00 = $120.00 → Not exceeded yet
Total discount applied = $110.00
Shipping waived (GOLD)
Tax = $17.60
Total = $200.00 - $110.00 + $17.60 = $107.60
Reason flags: ["membership_gold", "free_shipping"]




Case 6: SILVER + PCT50 + FLAT50 (Cap Enforced)

Inputs:

Items:

SKU: A1, Price: $100.00, Qty: 1


Quote:

Region: TX
Membership: SILVER
Coupon Codes: ["PCT50", "FLAT50"]




Expected Oracles:

Subtotal = $100.00
Raw discounts:

Membership: $2.00
PCT50: $50.00
FLAT50 ignored (stacking limit reached)
Total raw discount = $52.00


Cap = $60.00 → Not exceeded
Total discount applied = $52.00
Shipping = $5.99
Tax = $6.25
Total = $100.00 - $52.00 + $5.99 + $6.25 = $60.24
Reason flags: ["membership_silver", "stacking_limit_reached"]


**What happens:** You confirm revenue-protection cap.


## Task 5 — Stage Rounding
**Prompt:**  
"Confirm rounding at each stage (subtotal, discounts, tax, total)."


**Answer**  

Rounding at Each Stage


Subtotal:

subtotal = round_money(sum(it.price * it.qty for it in items))

Rounded to 2 decimals immediately after summing item prices.


Discounts:

Membership discount:

return round_money(subtotal * 0.05)  # GOLD

Coupon discounts (PCT, FLAT, BOGO):

return round_money(subtotal * pct)  # percentage
return round_money(min(amt, subtotal))  # flat
return round_money(discount)  # BOGO

Each discount is rounded individually before being applied.


Tax

tax_total += round_money(it.price * it.qty * rate)
tax_total = round_money(tax_total)

→ Per-item tax and total tax are rounded.


Total:

total = round_money(subtotal_after_discounts + shipping + tax_total)

Implications

Rounding happens at every stage, not just at the end.
This prevents fractional cents from accumulating but can cause slight discrepancies compared to rounding only at the final total.

**What happens:** You ensure deterministic currency math.


## Task 6 — BOGO on B1* SKUs
##### BOGO on B1* SKUs -> “Buy 1 Get 1 Free” — for SKUs starting with B1, every second item (one of each pair) is free.
**Prompt:**  
"Verify BOGO1 applies only to SKUs starting with B1."


**Answer**  

I checked the implementation of BOGO1 in promotion_pricing_engine.py. Here’s what I found:

Verification

The function _apply_bogo(items, code)

if code.upper() != "BOGO1":
    return 0.0
for it in items:
    if it.sku.startswith("B1"):
        free_units = it.qty // 2
        discount += free_units * it.price

Logic:

BOGO1 applies only if the coupon code is exactly "BOGO1".
Within that, it only discounts items whose SKU starts with "B1".
Discount = price × (qty // 2) → every second unit is free.

 Conclusion
Yes, BOGO1 is restricted to SKUs starting with B1. No other SKUs are affected.


**What happens:** You prove targeted BOGO behavior.


## Task 7 — Membership & Shipping Threshold
**Prompt:**  
"Check GOLD free ship always and SILVER threshold at 75 after discounts."


**Answer**  


GOLD Free Shipping

In _shipping_fee(region, membership, subtotal_after_discounts)

if membership == "GOLD" or subtotal_after_discounts >= CFG["free_ship_threshold"]:
    return 0.0

Result: GOLD membership always gets free shipping, regardless of subtotal.

SILVER Threshold

SILVER does not automatically waive shipping; it only gets a discount if subtotal ≥ $50 (for membership discount).
Free shipping for SILVER depends on:

subtotal_after_discounts >= CFG["free_ship_threshold"]  # 75.0

Result: SILVER gets free shipping only if subtotal after discounts ≥ $75.

Tests to Confirm


GOLD Free Shipping Always

Items subtotal $10, membership GOLD.
Expected: shipping=0.0, reason_flags includes "free_shipping".



SILVER Below Threshold

Items subtotal $74.99 after discounts, membership SILVER.
Expected: shipping > 0 (region-based fee), no "free_shipping" flag.



SILVER At Threshold

Items subtotal $75.00 after discounts, membership SILVER.
Expected: shipping=0.0, reason_flags includes "free_shipping".

**What happens:** You validate tier and threshold logic.


## Task 8 — Tax by Category & Exemption
**Prompt:**  
"Confirm food and tax-exempt SKUs untaxed; others taxed by region."


**Answer**  


Tax Rules Confirmed

In _tax_for_item(it, region):

if it.tax_exempt:
    return 0.0
if it.category == "food":
    return CFG["food_tax"]  # which is 0.0
return CFG["tax_by_region"].get(region, 0.0)

Behavior:

If tax_exempt=True → tax rate = 0.0.
If category="food" → tax rate = food_tax (configured as 0.0).
Otherwise → tax rate = region-specific rate (e.g., CA: 0.0825, NY: 0.088).

Implications

Food items and tax-exempt SKUs are always untaxed.
All other items are taxed based on region.


Suggested Tests


Food Item Untaxed

Item: sku="F123", category="food", price=10, region="CA".
Expected tax: $0.00.



Tax-Exempt Item Untaxed

Item: sku="X999", tax_exempt=True, price=10, region="CA".
Expected tax: $0.00.



General Item Taxed

Item: sku="G123", category="general", price=10, region="CA".
Expected tax: $0.83 (10 × 0.0825).



Mixed Basket

Food + tax-exempt + general item.
Expected: Only general item contributes to tax.

**What happens:** You verify taxation correctness.


## Task 9 — Minimum Payable Floor
**Prompt:**  
"Design a case driving total below $1 to trigger floor."


**Answer**  

Goal
Drive the final total below $1 after discounts and shipping/tax adjustments so the rule:

if total < 1.0 and subtotal > 0:
    reason_flags.append("min_payable_enforced")
    total = 1.0

is activated.

Inputs

Items

[
  {"sku": "B123", "price": 0.50, "qty": 2},  # Eligible for BOGO1
  {"sku": "X999", "price": 0.10, "qty": 1}
]

Region: "CA" (tax applies but negligible)
Membership: "GOLD" (adds discount + free shipping)
Coupons: ["BOGO1", "PCT10"] (stacking allowed up to 2)

Expected Flow


Subtotal:

(0.50 × 2) + (0.10 × 1) = $1.10 → rounded to $1.10.



Discounts:

Membership GOLD: 5% of $1.10 → $0.06.
BOGO1: One free unit of B123 → $0.50.
Total discount = $0.56 → subtotal after discounts = $0.54.



Shipping:

GOLD → free shipping.



Tax:

General item $0.10 taxed at 8.25% → $0.01.
Total tax ≈ $0.01.



Total before floor:

$0.54 + $0.00 + $0.01 = $0.55 (< $1).



Floor enforcement:

Total set to $1.00.
reason_flags includes "min_payable_enforced".

**What happens:** You confirm payment floor is enforced.


## Task 10 — Negative/Invalid Inputs
**Prompt:**  
"Document manual checks for invalid/edge inputs mirroring defensive unit tests."


**Answer**  

Negative or Zero Values

Check: price <= 0, qty <= 0, subtotal <= 0.
Why: Prevent nonsensical totals or free items.
Manual Action: Enter an item with price=0 or qty=0 and confirm subtotal remains valid or triggers error.


2. Invalid Coupon Codes

Check: Nonexistent codes or malformed strings (e.g., "PCTX", "FLAT-5").
Why: Ensure _apply_coupon returns 0.0 and does not crash.
Manual Action: Apply invalid code and confirm no discount applied.


3. Excessive Discounts

Check: Discounts exceeding 60% cap.
Why: Validate discount_cap_applied flag and capped total.
Manual Action: Apply multiple high-value coupons and confirm cap enforcement.


4. Stacking Limit

Check: More than 2 discounts applied.
Why: Ensure stacking_limit_reached flag appears and extra codes ignored.
Manual Action: Apply 3+ coupons and confirm only 2 applied.


5. Extreme Subtotal

Check: Very large subtotal (e.g., $1,000,000) or very small subtotal (e.g., $0.01).
Why: Validate rounding and min payable logic.
Manual Action: Confirm totals round correctly and enforce $1 floor when needed.


6. Tax Edge Cases

Check: Food items and tax-exempt SKUs.
Why: Ensure tax = 0 for these categories.
Manual Action: Mix food, tax-exempt, and general items; verify tax breakdown.


7. Shipping Logic

Check: GOLD always free; SILVER free only if subtotal after discounts ≥ $75.
Manual Action: Test SILVER at $74.99 and $75.00 after discounts.


8. Collateral/Membership Nulls

Check: membership=None, empty coupon list.
Why: Ensure no crash and defaults apply.
Manual Action: Quote with no membership and no coupons.


9. Rounding Consistency

Check: Subtotal, discounts, tax, total all rounded to 2 decimals.
Manual Action: Use fractional prices (e.g., $9.995) and confirm rounding at each stage. 

**What happens:** You ensure robustness against bad inputs.
